### Pertama, kita panggil terlebih dahulu pustaka-pustaka yang dibutuhkan dan data pembelajaran yang sudah kita persiapkan sebelumnya:

In [5]:
import pickle
import spacy
import random
from spacy.util import minibatch, compounding
from spacy import load, displacy

### Pemanggilan data pembelajaran

In [6]:
with open('ner_spacy_fmt_datasets.pickle', 'rb') as f:
    ner_spacy_fmt_datasets = pickle.load(f)

### Selanjutnya kita akan buat model dari “model kosong” atau “blank-model” dengan perintah berikut

In [3]:
nlp=spacy.blank("id")

nlp.add_pipe(nlp.create_pipe('ner'))

nlp.begin_training()

/home/wolio/code/event-extraction/venv/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  proc.begin_training(


### Kita ambil data-modul untuk NER dan menyisihkan yang lainnya.

In [4]:
ner = nlp.get_pipe("ner")

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

### Kita proses atau ambil label tipe entitas dari data training

In [5]:
for _, annotations in ner_spacy_fmt_datasets:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])
        break

#### Untuk data train akan kita random terlebih dahulu baru kemudian kita masukkan ke iterasi training, pada contoh ini kita coba untuk latih model sebanyak 30 kali perulangan.

In [6]:
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(ner_spacy_fmt_datasets)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(ner_spacy_fmt_datasets, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
    
    print("Losses at iteration {}".format(iteration), losses)

Losses at iteration 0 {'ner': 46313.4103863339}
Losses at iteration 1 {'ner': 44726.05711008631}
Losses at iteration 2 {'ner': 44688.753690799625}
Losses at iteration 3 {'ner': 44609.0059457256}
Losses at iteration 4 {'ner': 44404.945039788014}
Losses at iteration 5 {'ner': 44353.42911557443}
Losses at iteration 6 {'ner': 43728.91765047358}
Losses at iteration 7 {'ner': 43858.77372805169}
Losses at iteration 8 {'ner': 43789.24379751411}
Losses at iteration 9 {'ner': 43434.64601052977}
Losses at iteration 10 {'ner': 43372.82921985403}
Losses at iteration 11 {'ner': 43306.30348470662}
Losses at iteration 12 {'ner': 43309.700816553064}
Losses at iteration 13 {'ner': 43317.673493076916}
Losses at iteration 14 {'ner': 43432.562191384226}
Losses at iteration 15 {'ner': 43086.468654191995}
Losses at iteration 16 {'ner': 43155.663107672044}
Losses at iteration 17 {'ner': 43164.73287854719}
Losses at iteration 18 {'ner': 43117.04583458355}
Losses at iteration 19 {'ner': 42849.37611311673}
Losse

In [7]:
# test 
doc = nlp("SELUBUNG yang menyelimuti kasus penembakan yang menewaskan Pendeta Yeremia Zanambani di Kabupaten Intan Jaya, Papua kian terkuak. Hasil investigasi Tim Gabungan Pencari Fakta (TGPF) kasus tersebut menyatakan bahwa penembakan di Intan Jaya diduga dilakukan oleh aparat keamanan.")

print(doc.ents)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

(SELUBUNG, Pendeta, Yeremia Zanambani, Kabupaten Intan Jaya, Papua, Intan Jaya)
Entities [('SELUBUNG', 'ORGANIZATION'), ('Pendeta', 'ORGANIZATION'), ('Yeremia Zanambani', 'PERSON'), ('Kabupaten Intan Jaya', 'LOCATION'), ('Papua', 'LOCATION'), ('Intan Jaya', 'LOCATION')]


## Menyimpan model

In [8]:
from pathlib import Path

output_dir = Path('nlp_id_checkpoint_2021_11_02')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to nlp_id_checkpoint_2021_11_02


In [9]:
# load existing model 
output_dir = 'nlp_id_checkpoint_2021_11_02'
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)

Loading from nlp_id_checkpoint_2021_11_02


In [10]:
# contoh 1
doc = nlp_updated("Kementerian Perhubungan tidak mewajibkan rapid test COVID-19 untuk perjalanan darat lintas daerah, kecuali untuk tujuan Bali. Termasuk, dalam periode cuti bersama." )

print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

displacy.render(doc, style="ent")

Entities [('Bali', 'LOCATION')]


In [11]:
# contoh 2
doc = nlp_updated("Empat saksi terkait korupsi proyek infrastruktur fiktif yang dikerjakan PT Waskita Karya (Persero) Tbk absen dari panggilan KPK hari ini. Seorang di antaranya mantan Bupati Wakatobi, Hugua.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

displacy.render(doc, style="ent")

Entities [('Empat saksi', 'QUANTITY'), ('PT Waskita Karya (Persero) Tbk', 'ORGANIZATION'), ('KPK', 'ORGANIZATION'), ('Hugua', 'PERSON')]


In [3]:
# load existing model 
output_dir = 'nlp_id_checkpoint_2021_11_02'
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)

Loading from nlp_id_checkpoint_2021_11_02


In [4]:
doc = nlp_updated("Satu hari sebelum saya bertemu dengan Gung Kayon, Presiden Joko Widodo gembar-gembor ingin masyarakat untuk tinggalkan energi fosil dan mulai beralih ke energi terbarukan. Mungkin Jokowi tak sadar, apa yang diucapkannya sudah dijalani dan dihayati oleh Gung Kayon sejak puluhan tahun silam." )

print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

displacy.render(doc, style="ent")

Entities [('Gung Kayon', 'PERSON'), ('Joko Widodo', 'PERSON'), ('Jokowi', 'PERSON'), ('Gung Kayon', 'PERSON')]
